##### <p> Samuel Wolfe <br> September 24, 2023 <br> MSBA 207 <br> SBA Case: Logistic Regression </p>

In [1]:
%matplotlib inline
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf
# pip install mord
from mord import LogisticIT
import matplotlib.pylab as plt
import seaborn as sns
from dmba import classificationSummary, gainsChart, liftChart
from dmba.metric import AIC_score
import math

keepers = ['Selected','Default','New','RealEstate','DisbursementGross','Portion','Recession']

DATA = Path('E:/Aliit/School/MSBA/206/MSBA-206/dmba')

loan_df = pd.read_csv(DATA / 'SBAcase.11.13.17.csv')
loan_df = loan_df[keepers]
loan_df

,Selected,Default,New,RealEstate,DisbursementGross,Portion,Recession
0,0,0,0,0,32812,0.50,0
1,1,0,0,0,30000,0.50,1
2,0,0,0,0,30000,0.50,0
3,1,0,0,0,50000,0.50,0
4,1,0,0,1,343000,1.00,0
...,...,...,...,...,...,...,...
2097,1,0,1,0,150000,0.50,0
2098,1,0,0,1,99000,0.80,0
2099,0,0,0,0,50000,0.80,0
2100,1,0,0,0,251150,0.75,0


In [2]:
y = loan_df['Default']
X = loan_df.drop(columns=['Default'])

# partition data
train_X = X.where(loan_df['Selected'] == 1).dropna().drop(columns='Selected')
valid_X = X.where(loan_df['Selected'] == 0).dropna().drop(columns='Selected')
train_y = y.where(loan_df['Selected'] == 1).dropna().drop(columns='Selected')
valid_y = y.where(loan_df['Selected'] == 0).dropna().drop(columns='Selected')
X = X.drop(columns='Selected')

## (a) I hope below this is the correct output. I could not find anything that would produce explicitly the output generated in the SBA case.

In [3]:
train_X2 = sm.add_constant(train_X, prepend=True)
logit_full1 = sm.GLM(train_y, train_X2, family=sm.families.Binomial())
logit_result1 = logit_full1.fit()
print('Table 7')
logit_result1.summary()

Table 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                Default   No. Observations:                 1051
Model:                            GLM   Df Residuals:                     1045
Model Family:                Binomial   Df Model:                            5
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -540.85
Date:                Mon, 02 Oct 2023   Deviance:                       1081.7
Time:                        18:11:53   Pearson chi2:                 1.00e+03
No. Iterations:                     6   Pseudo R-squ. (CS):             0.1949
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 1.3537      0.323      4.192      0.000       0.721       1.987
New                  -0.0772      0.210     -0.367      0.713      -0.489       0.335
RealEstate           -2.0331      0.364     -5.592      0.000      -2.746      -1.320
DisbursementGross -3.371e-07   3.52e-07     -0.958      0.338   -1.03e-06    3.53e-07
Portion              -2.8298      0.559     -5.059      0.000      -3.926      -1.733
Recession             0.4971      0.241      2.060      0.039       0.024       0.970
=====================================================================================
"""

In [4]:
keepers = ['Selected','Default','RealEstate','Portion','Recession']

DATA = Path('E:/Aliit/School/MSBA/206/MSBA-206/dmba')

loan_df = pd.read_csv(DATA / 'SBAcase.11.13.17.csv')
loan_df = loan_df[keepers]
y = loan_df['Default']
X = loan_df.drop(columns=['Default'])

# partition data
train_X = X.where(loan_df['Selected'] == 1).dropna().drop(columns='Selected')
valid_X = X.where(loan_df['Selected'] == 0).dropna().drop(columns='Selected')
train_y = y.where(loan_df['Selected'] == 1).dropna().drop(columns='Selected')
valid_y = y.where(loan_df['Selected'] == 0).dropna().drop(columns='Selected')
X = X.drop(columns='Selected')

In [5]:
train_X2 = sm.add_constant(train_X, prepend=True)
logit_full2 = sm.GLM(train_y, train_X2, family=sm.families.Binomial())
logit_result2 = logit_full2.fit()
print('Table 8')
logit_result2.summary()

Table 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                Default   No. Observations:                 1051
Model:                            GLM   Df Residuals:                     1047
Model Family:                Binomial   Df Model:                            3
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -541.38
Date:                Mon, 02 Oct 2023   Deviance:                       1082.8
Time:                        18:11:53   Pearson chi2:                     989.
No. Iterations:                     6   Pseudo R-squ. (CS):             0.1941
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.3931      0.322      4.332      0.000       0.763       2.023
RealEstate    -2.1282      0.345     -6.169      0.000      -2.804      -1.452
Portion       -2.9875      0.539     -5.540      0.000      -4.044      -1.931
Recession      0.5041      0.241      2.090      0.037       0.031       0.977
==============================================================================
"""

## (b) The indicators in Table 8, shown above, were chosen because the P-values of 'New' and 'DisbursementGross' were well above the threshold of 0.05, shown above in Table 7, making those indicators not significantly significant to the model.

In [6]:
valid_X2 = sm.add_constant(valid_X, prepend=True)
logit = sm.GLM(train_y, train_X2, family=sm.families.Binomial())
result = logit.fit()
predictions = result.predict(valid_X2)
predictions_nominal = [ 0 if x < 0.5 else 1 for x in predictions]
logit_result = pd.DataFrame({'actual': valid_y, 
                             'p(0)': 1 - predictions,
                             'p(1)': predictions,
                             'predicted': predictions_nominal,
                             'backed': valid_X['RealEstate'],
                             'recession': valid_X['Recession'],
                             'SBA': valid_X['Portion']})
print(result.params)
print(logit_result)

const         1.393064
RealEstate   -2.128217
Portion      -2.987482
Recession     0.504123
dtype: float64
      actual      p(0)      p(1)  predicted  backed  recession   SBA
0        0.0  0.525148  0.474852          0     0.0        0.0  0.50
2        0.0  0.525148  0.474852          0     0.0        0.0  0.50
6        1.0  0.951467  0.048533          0     1.0        0.0  0.75
7        0.0  0.525148  0.474852          0     0.0        0.0  0.50
8        1.0  0.525148  0.474852          0     0.0        0.0  0.50
...      ...       ...       ...        ...     ...        ...   ...
2094     0.0  0.525148  0.474852          0     0.0        0.0  0.50
2095     0.0  0.976400  0.023600          0     1.0        0.0  1.00
2096     0.0  0.976400  0.023600          0     1.0        0.0  1.00
2099     0.0  0.730453  0.269547          0     0.0        0.0  0.80
2101     0.0  0.525148  0.474852          0     0.0        0.0  0.50

[1051 rows x 7 columns]


In [7]:
#Real Estate Backing
print(logit_result.sort_values(by='backed', ascending=False).where(logit_result['backed'] >= 0.0).where(logit_result['predicted'] >= 0.0).head(5))
print('Odds of default with backing: ', round(-2.128217,4))

      actual      p(0)      p(1)  predicted  backed  recession   SBA
1040     0.0  0.963544  0.036456          0     1.0        0.0  0.85
588      0.0  0.951467  0.048533          0     1.0        0.0  0.75
680      0.0  0.976400  0.023600          0     1.0        0.0  1.00
674      0.0  0.951467  0.048533          0     1.0        0.0  0.75
669      0.0  0.976400  0.023600          0     1.0        0.0  1.00
Odds of default with backing:  -2.1282


In [8]:
#Recession
print(logit_result.sort_values(by='recession', ascending=False).where(logit_result['recession'] >= 0.0).where(logit_result['predicted'] >= 0.0).head(5))
print('Odds of default during recession: ', round(0.504123,4))

      actual      p(0)      p(1)  predicted  backed  recession  SBA
1574     0.0  0.400485  0.599515          1     0.0        1.0  0.5
654      1.0  0.400485  0.599515          1     0.0        1.0  0.5
381      1.0  0.400485  0.599515          1     0.0        1.0  0.5
1729     1.0  0.400485  0.599515          1     0.0        1.0  0.5
1728     1.0  0.400485  0.599515          1     0.0        1.0  0.5
Odds of default during recession:  0.5041


In [9]:
#SBA Guarantee
print(logit_result.sort_values(by='SBA', ascending=False).where(logit_result['SBA'] >= 0.0).where(logit_result['predicted'] >= 0.0).head(5))
print('Odds of default based on SBA Backing proportion: ', round(-2.987482,4))

      actual    p(0)    p(1)  predicted  backed  recession  SBA
501      0.0  0.9764  0.0236          0     1.0        0.0  1.0
660      0.0  0.9764  0.0236          0     1.0        0.0  1.0
1025     0.0  0.9764  0.0236          0     1.0        0.0  1.0
648      0.0  0.9764  0.0236          0     1.0        0.0  1.0
1917     0.0  0.9764  0.0236          0     1.0        0.0  1.0
Odds of default based on SBA Backing proportion:  -2.9875


## (c)
###    (i) A loan backed by real estate is less likely to default. Its Odds value is -2.1282.
###    (ii) A loan given during the recession is more likely to default. Its odds value is 0.5041.
###    (iii)  A loan that has SBA Guarantee portions are less likely to default. its odds value is -2.9875.

In [14]:
math.exp(-2.1282)

0.11905139361332186

In [15]:
math.exp(0.5041)

1.6554949043702933

In [16]:
math.exp(-2.9875)

0.05041331259467938

In [10]:
valid_X

,RealEstate,Portion,Recession
0,0.0,0.50,0.0
2,0.0,0.50,0.0
6,1.0,0.75,0.0
7,0.0,0.50,0.0
8,0.0,0.50,0.0
...,...,...,...
2094,0.0,0.50,0.0
2095,1.0,1.00,0.0
2096,1.0,1.00,0.0
2099,0.0,0.80,0.0


In [11]:
testData = pd.DataFrame({'RealEstate': [1.0,0.0],
                         'Portion': [0.75,0.40],
                         'Recession': [0.0,0.0]})
testData
testData2 = sm.add_constant(testData, prepend=True)
testData2

,const,RealEstate,Portion,Recession
0,1.0,1.0,0.75,0.0
1,1.0,0.0,0.40,0.0


In [12]:
predictions2 = result.predict(testData2)
predictions_nominal = [ 0 if x < 0.5 else 1 for x in predictions2]
logit_result2 = pd.DataFrame({'Estimated probability of Default': round(predictions2,2),
                             'Approved == 0': predictions_nominal})

print(logit_result2)

   Estimated probability of Default  Approved == 0
0                              0.05              0
1                              0.55              1


## (d) As you can see for column 'Estimated probability of Default' my numbers match Table 10 from the SBA article. As well my model accurately predicted approval in column 'Approved == 0'. Granted the numbers were flipped.